In [ ]:
# !pip install --upgrade pip
# !pip install tensorflow tfrecorder

In [ ]:
# !pip install --upgrade avro-python3==1.9.2 tensorflow-transform==0.24

In [ ]:
!pip install modin[dask] # Install all of the above


In [ ]:
import os
import glob
import base64

import pandas as pd
import numpy as np
import tfrecorder
from tfrecorder import input_schema
from tfrecorder import types

def convert_tsv_tfrecord(path, to_dir, schema, dil='\t'):

  def labels_decode(x):
     return np.array2string(np.frombuffer(base64.b64decode(x['class_labels']), dtype=np.int64).reshape(x['num_boxes']))
  
  def box_decode(x):
    return np.array2string(np.frombuffer(base64.b64decode(x['boxes']), dtype=np.float32).reshape(x['num_boxes'], 4))

  def features_decode(x):
    return np.array2string(np.frombuffer(base64.b64decode(x['features']), dtype=np.float32).reshape(x['num_boxes'], 2048))


  df = pd.read_csv(path, sep=dil)
  df['split'] = 'TRAIN'
  df['label'] = 1


  df.tensorflow.to_tfr(to_dir,schema=schema, )

In [ ]:


TRAIN_PATH_TSV = "/root/Applied_AI_Lab_WiSe2021_Passau/data/sample/train.sample.tsv"
TRAIN_DIR_TFRECORDS = "/data/sample/tfrecord/train/"

csv_schema = input_schema.Schema({
  
    'features': types.StringInput,
    'class_labels': types.StringInput,
    'boxes': types.StringInput,
    'product_id': types.IntegerInput,
    'image_h': types.IntegerInput,
    'image_w': types.IntegerInput,
    'num_boxes': types.IntegerInput,
    'query_id': types.IntegerInput,
    'query': types.StringInput,
    'split': types.SplitKey,
    'label': types.IntegerLabel,
})


In [ ]:
convert_tsv_tfrecord(TRAIN_PATH_TSV, TRAIN_DIR_TFRECORDS, csv_schema)

## Convert TSV to CSV

In [ ]:
# !python -m pip install "dask[dataframe]" --upgrade

In [ ]:
import dask.dataframe as dd
import pandas as pd
import csv
import base64

import numpy as np

In [ ]:
import csv
df = dd.read_csv('/../home/data/train.tsv', sep='\t', quoting=csv.QUOTE_NONE, encoding='utf-8',
error_bad_lines=False)

In [ ]:
df.to_csv('train/real/train-*.csv', index=False)

In [ ]:
import csv
df = dd.read_csv('/root/Applied_AI_Lab_WiSe2021_Passau/sample_train/train.sample.tsv', sep='\t', quoting=csv.QUOTE_NONE, encoding='utf-8',
error_bad_lines=False)
df1 = dd.read_csv('/root/Applied_AI_Lab_WiSe2021_Passau/eval/valid.tsv', sep='\t', quoting=csv.QUOTE_NONE, encoding='utf-8',
error_bad_lines=False)
df2 = dd.read_csv('/root/Applied_AI_Lab_WiSe2021_Passau/testA/testA.tsv', sep='\t', quoting=csv.QUOTE_NONE, encoding='utf-8',
error_bad_lines=False)
df3 = dd.read_csv('/root/Applied_AI_Lab_WiSe2021_Passau/testB/testB.tsv', sep='\t', quoting=csv.QUOTE_NONE, encoding='utf-8',
error_bad_lines=False)

In [ ]:
df.to_csv('/root/Applied_AI_Lab_WiSe2021_Passau/sample_train/real/real.csv', index=False, single_file=True)
df1.to_csv('/root/Applied_AI_Lab_WiSe2021_Passau/eval/real/real.csv', index=False, single_file=True)
df2.to_csv('/root/Applied_AI_Lab_WiSe2021_Passau/test/testA/real/real.csv', index=False, single_file=True)
df3.to_csv('/root/Applied_AI_Lab_WiSe2021_Passau/test/testB/real/real.csv', index=False, single_file=True)

### Decode base64

In [ ]:
df = pd.read_csv('/root/Applied_AI_Lab_WiSe2021_Passau/sample_train/train.sample.tsv', sep='\t', quoting=csv.QUOTE_NONE,
error_bad_lines=False)

In [ ]:
def decode_base64_boxes(x):
    
    return np.frombuffer(base64.b64decode(x['boxes']), dtype=np.float32).reshape(x['num_boxes'], 4)
 
def decode_base64_features(x):
    
    return np.frombuffer(base64.b64decode(x['features']), dtype=np.float32).reshape(x['num_boxes'], 2048)

def decode_base64_labels(x):
    return np.frombuffer(base64.b64decode(x['class_labels']), dtype=np.int64).reshape(x['num_boxes'])


In [ ]:
df['boxes'] = df.apply(decode_base64_boxes, axis=1)
df['features'] = df.apply(decode_base64_features, axis=1)
df['class_labels'] = df.apply(decode_base64_labels, axis=1)
df.to_csv('/root/Applied_AI_Lab_WiSe2021_Passau/sample_train/real/real.csv', index=False)
